# Método Numérico de Numerov

* PET - Física UFRN
* Petiano: José Arthur de Luna Oliveira
* Data:

## 1. Introdução

# Referências

**[1]** Kalogiratou, Z., et al. “Numerical Solution of the Two-Dimensional Time Independent Schrödinger Equation with Numerov-Type Methods.” Journal of Mathematical Chemistry, vol. 37, no. 3, Apr. 2005, pp. 271–279, https://doi.org/10.1007/s10910-004-1469-1. Accessed 29 Mar. 2024.

**[2]** Investigação da energia de ligação entre átomos utilizando o potencial de Lennard-Jones: https://github.com/Josearthur266/PET.py/blob/main/Notebooks/Investigação%20da%20energia%20de%20ligação%20entre%20átomos%20utilizando%20o%20potencial%20de%20Lennard-Jones..ipynb

**[3]** Hjorth-Jensen, Morten. Computational Physics. University of Oslo, Aug. 2014.

**[4]** Caruso, Francisco, et al. “Applications of the Numerov Method to Simple Quantum Systems Using Python.” Revista Brasileira de Ensino de Física, vol. 44, no. e20220098, 2022, https://doi.org/10.1590/1806-9126-rbef-2022-0098. Accessed 29 Mar. 2024.